<a href="https://colab.research.google.com/github/abhinav1401/Implementation-of-Decision-Tree-Classifer-Algorithm/blob/main/Data_Analytics_13313162.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import Required Python Libraries



In [ ]:
import pandas as pd
import numpy as np


#Data Prepration

In [ ]:
#Loading The Dataset

df = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/zoo/zoo.data',
                names=['animal_name','hair','feathers','eggs','milk','airbone','aquatic','predator','toothed',
                        'backbone','breathes','venomous','fins','legs','tail','domestic','catsize','Animal_Type',])


In [ ]:
#Drop The Unnecessary Columns

df = df.drop('animal_name',axis=1)


In [ ]:
#Printing The First 10 Rows of The Dataset

print(df.head(10))

   hair  feathers  eggs  milk  ...  tail  domestic  catsize  Animal_Type
0     1         0     0     1  ...     0         0        1            1
1     1         0     0     1  ...     1         0        1            1
2     0         0     1     0  ...     1         0        0            4
3     1         0     0     1  ...     0         0        1            1
4     1         0     0     1  ...     1         0        1            1
5     1         0     0     1  ...     1         0        1            1
6     1         0     0     1  ...     1         1        1            1
7     0         0     1     0  ...     1         1        0            4
8     0         0     1     0  ...     1         0        0            4
9     1         0     0     1  ...     0         1        0            1

[10 rows x 17 columns]


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101 entries, 0 to 100
Data columns (total 17 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   hair         101 non-null    int64
 1   feathers     101 non-null    int64
 2   eggs         101 non-null    int64
 3   milk         101 non-null    int64
 4   airbone      101 non-null    int64
 5   aquatic      101 non-null    int64
 6   predator     101 non-null    int64
 7   toothed      101 non-null    int64
 8   backbone     101 non-null    int64
 9   breathes     101 non-null    int64
 10  venomous     101 non-null    int64
 11  fins         101 non-null    int64
 12  legs         101 non-null    int64
 13  tail         101 non-null    int64
 14  domestic     101 non-null    int64
 15  catsize      101 non-null    int64
 16  Animal_Type  101 non-null    int64
dtypes: int64(17)
memory usage: 13.5 KB


#Data Splitting 

In [ ]:
#Function to split the data into training and testing sets

def train_test_split(df):
    training_data = df.iloc[:90].reset_index(drop=True) #90% rows as training data
    testing_data = df.iloc[30:].reset_index(drop=True) #30% rows as testing data
    return training_data,testing_data

training_data = train_test_split(df)[0]
testing_data = train_test_split(df)[1]

#Entropy

In [ ]:
#Function to Calculate the Entropy

def calculate_entropy(target_column):
    histogram = np.bincount(target_column)
    ps = histogram / len(target_column)
    result= -np.sum([p * np.log2(p) for p in ps if p > 0])
    return result

#Information Gain

In [ ]:
#Function to Calculate the Information Gain

def Information_Gain(data,split_attribute_name,target_name="Animal_Type"):

    #Calculation of the Total Entropy
    total_entropy = calculate_entropy(data[target_name])
    
    #Calculation of Split Counts and Values for the Attribure
    values,counts= np.unique(data[split_attribute_name],return_counts=True)

    #Calculation of Weighted Entropy
    weighted_entropyropy = np.sum([(counts[i]/np.sum(counts))*calculate_entropy(data.where(data[split_attribute_name]==values[i]).dropna()[target_name]) for i in range(len(values))])
    
    #Calculation of Information Gain
    Information_Gain = total_entropy - weighted_entropyropy
    
    return Information_Gain

#Decision Tree Algorithm

In [ ]:
#ID3 Decision Tree Algorithm

def decision_tree_algorithm(data,original_data,features,target_attribute_name="Animal_Type",parent_node_class = None):

    # Statement to terminate the algorithm and return the leaf node
    if len(np.unique(data[target_attribute_name])) <= 1:
        return np.unique(data[target_attribute_name])[0]

    #Returns the feature value from the original data if the training data is no row of data
    elif len(data)==0:
        return np.unique(original_data[target_attribute_name])[np.argmax(np.unique(original_data[target_attribute_name],return_counts=True)[1])]

    #Returns the parent node if the current node has no features
    elif len(features) ==0:
        return parent_node_class

    #Tree Building Statements
    else:
        parent_node_class = np.unique(data[target_attribute_name])[np.argmax(np.unique(data[target_attribute_name],return_counts=True)[1])]

        #Calculating the best split using information gain
        item_values = [Information_Gain(data,feature,target_attribute_name) for feature in features]

        #Calculating Information Gain and Features
        best_feature_index = np.argmax(item_values)
        best_feature = features[best_feature_index]

        #Tree Creation
        tree = {best_feature:{}}

        features = [i for i in features if i != best_feature]

        #Recursive Tree
        for value in np.unique(data[best_feature]):
            value = value
            sub_data = data.where(data[best_feature] == value).dropna()
            subtree = decision_tree_algorithm(sub_data,df,features,target_attribute_name,parent_node_class)
            tree[best_feature][value] = subtree

        return(tree)

#Prediction & Testing

In [ ]:
#Function for Prediction

def predict_data(query,tree,default = 1):

    #Match each feature in query
    for key in list(query.keys()):
        if key in list(tree.keys()):

            try:
                result = tree[key][query[key]]
            except:
                return default

            result = tree[key][query[key]]

            if isinstance(result,dict):
                return predict_data(query,result)

            else:
                return result

In [ ]:
#Function to test the data

def test_data(data,tree):

    #Converting the targeted column in dictionary format
    queries = data.iloc[:,:-1].to_dict(orient = "records")

    # To store the predicted data
    predicted = pd.DataFrame(columns=["predicted"])

    #To calculate the accuracy of the prediction
    for i in range(len(data)):
        predicted.loc[i,"predicted"] = predict_data(queries[i],tree,1.0)
    print('The Accuracy of Algorithm is : ',(np.sum(predicted["predicted"] == data["Animal_Type"])/len(data))*100,'%')

#Run Decision Tree Algorithm 

In [ ]:
tree = decision_tree_algorithm(training_data,training_data,training_data.columns[:-1])
test_data(testing_data,tree)

The Accuracy of Algorithm is :  97.1830985915493 %
